# LangChain Agent with MCP Server Integration

## Overview

In this tutorial, you will learn how to build and deploy a distributed AI agent system using:
- **MCP Server**: A Model Context Protocol server providing utility tools
- **LangChain Agent**: An agent that consumes and orchestrates MCP server tools
- **AgentRun CLI**: To deploy both components separately to local Kubernetes

This demonstrates a microservices architecture where the tool provider (MCP server) and the agent orchestrator (LangChain agent) are deployed and scaled independently.

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Advanced Integration                                                         |
| Agent type          | LangChain Agent + MCP Server                                                 |
| Framework           | LangChain + MCP + FastAPI                                                    |
| Language            | Python 3.8+                                                                  |
| Tutorial components | MCP server creation, LangChain integration, distributed deployment           |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Intermediate                                                                 |
| Tools used          | AgentRun CLI, Docker, Kubernetes, LangChain, MCP                             |

### What You'll Learn

* How to create an MCP server with multiple useful tools
* How to integrate MCP servers with LangChain agents
* How to deploy distributed agent systems using AgentRun CLI
* How agents and tools can communicate across services
* Best practices for microservices-based AI architectures

### Tutorial Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    User Request                             │
│                         │                                   │
│                         ▼                                   │
│            ┌────────────────────────┐                       │
│            │  LangChain Agent       │                       │
│            │  (Port 8080)           │                       │
│            │                        │                       │
│            │  - Receives prompts    │                       │
│            │  - Orchestrates tools  │                       │
│            │  - Returns responses   │                       │
│            └────────────┬───────────┘                       │
│                         │                                   │
│                         │ HTTP/MCP                          │
│                         ▼                                   │
│            ┌────────────────────────┐                       │
│            │  MCP Server            │                       │
│            │  (Port 8000)           │                       │
│            │                        │                       │
│            │  Tools:                │                       │
│            │  - Calculator          │                       │
│            │  - File Operations     │                       │
│            │  - DateTime Utils      │                       │
│            └────────────────────────┘                       │
│                                                              │
│         Both deployed separately to Kubernetes              │
└─────────────────────────────────────────────────────────────┘
```

### Key Features

* **Distributed Architecture**: Agent and tools deployed separately
* **MCP Protocol**: Standard protocol for tool communication
* **LangChain Integration**: Seamless integration with LangChain ecosystem
* **Multiple Tool Categories**: Calculator, file operations, and datetime utilities
* **Production-Ready**: Deployed to Kubernetes with proper service discovery

## Prerequisites

Before starting this tutorial, ensure you have:

* **Python 3.8+** installed
* **Docker** installed and running
* **Kubernetes** (Docker Desktop with Kubernetes enabled, or minikube)
* **AgentRun CLI** installed
* Basic understanding of:
  - Python programming
  - REST APIs
  - LangChain agents (recommended)
  - Model Context Protocol (MCP) basics (recommended)

## Step 1: Install Dependencies

First, let's install AgentRun CLI and verify it's working.

In [1]:
# Install AgentRun CLI
%pip install -e ../ --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Verify installation
!agentrun --version

AgentRun CLI version: 0.1.0


## Step 2: Explore the MCP Server

Let's examine the MCP server that provides utility tools.

In [3]:
# View MCP server structure
!ls -la mcp_server/

total 32
drwxr-xr-x  4 ym  staff   128 Nov 26 09:44 .
drwxr-xr-x  8 ym  staff   256 Nov 26 09:44 ..
-rw-r--r--  1 ym  staff  8215 Nov 26 09:44 main.py
-rw-r--r--  1 ym  staff    73 Nov 26 09:44 requirements.txt


In [4]:
# View MCP server tools (first 100 lines)
!head -100 mcp_server/main.py

"""
MCP Server with Utility Tools

This MCP server provides a collection of useful tools:
- Calculator: Perform mathematical calculations
- File operations: Read, write, and list files in a workspace
- DateTime utilities: Get current time, format dates, timezone conversions

The server uses FastMCP with stateless HTTP transport for compatibility
with AgentRun CLI deployment.
"""

import os
import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Optional
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP(
    name="utility-tools-server",
    host="0.0.0.0",
    port=8000,
    stateless_http=True
)

# Create a workspace directory for file operations
WORKSPACE_DIR = Path("/tmp/mcp_workspace")
WORKSPACE_DIR.mkdir(exist_ok=True)


# ============================================================================
# Calculator Tools
# ============================================================================

@mcp.tool()
def

### Understanding the MCP Server

The MCP server provides **8 utility tools** across three categories:

**Calculator Tools:**
1. `calculate`: Evaluate mathematical expressions (e.g., "2 + 2", "10 * 5")
2. `power`: Calculate base raised to exponent (e.g., power(2, 3) = 8)

**File Operation Tools:**
3. `write_file`: Write content to a file in the workspace
4. `read_file`: Read content from a file
5. `list_files`: List all files in the workspace

**DateTime Tools:**
6. `get_current_time`: Get current date and time (with timezone support)
7. `format_timestamp`: Convert Unix timestamp to formatted date string

**Info Tools:**
8. `server_info`: Get MCP server information

The server uses:
- **FastMCP**: Modern MCP server framework
- **Stateless HTTP**: For compatibility with AgentRun deployment
- **Port 8000**: Standard MCP server port

## Step 3: Explore the LangChain Agent

Now let's examine the LangChain agent that will consume the MCP server tools.

In [5]:
# View agent structure
!ls -la agent/

total 40
drwxr-xr-x  5 ym  staff    160 Nov 26 09:44 .
drwxr-xr-x  8 ym  staff    256 Nov 26 09:44 ..
-rw-r--r--  1 ym  staff    238 Nov 26 09:44 .env.example
-rw-r--r--  1 ym  staff  12133 Nov 26 09:44 main.py
-rw-r--r--  1 ym  staff    142 Nov 26 09:44 requirements.txt


In [6]:
# View agent code (first 80 lines)
!head -80 agent/main.py

"""
LangChain Agent with MCP Server Integration

This agent demonstrates how to integrate LangChain with an MCP server
to provide enhanced tool-calling capabilities.

The agent:
- Connects to an MCP server via HTTP transport
- Converts MCP tools to LangChain tools using langchain-mcp-adapters
- Provides a FastAPI endpoint for user interaction
- Can be deployed separately from the MCP server

This showcases a microservices architecture for AI agents where:
- MCP server provides the tools
- LangChain agent orchestrates the tools
- Both can be deployed and scaled independently
"""

import os
import asyncio
from typing import Optional, Dict, Any, List
from datetime import timedelta

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool

# MCP 

### Understanding the LangChain Agent

The LangChain agent:
- **Connects to MCP Server**: Uses MCP client to discover and call tools
- **Converts MCP to LangChain**: Transforms MCP tools into LangChain-compatible tools
- **Orchestrates Tool Calls**: Uses LangChain's ReAct agent to decide when and how to use tools
- **Exposes REST API**: Provides FastAPI endpoints for interaction

Key features:
- **Mock Mode**: Works without OpenAI API key for testing
- **Production Mode**: Uses OpenAI for real agent reasoning (when API key is set)
- **Service Discovery**: Automatically connects to MCP server via environment variables
- **Health Checks**: Provides health and status endpoints

## Step 4: Test MCP Server Locally (Optional)

Before deploying, let's test the MCP server locally to ensure it works.

In [7]:
# Install MCP server dependencies
%pip install -r mcp_server/requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [28]:
# Start MCP server in background
import subprocess
import time

mcp_proc = subprocess.Popen(
    ['python', 'mcp_server/main.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(3)
print(f"MCP Server started (PID: {mcp_proc.pid})")
print("Server running on http://localhost:8000")

MCP Server started (PID: 26025)
Server running on http://localhost:8000


In [29]:
# Test MCP server with a simple client
import asyncio
from datetime import timedelta
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def test_mcp_server():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}
    
    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=30), terminate_on_close=False) as (
        read_stream, write_stream, _
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            
            # List available tools
            tool_result = await session.list_tools()
            print("\n📋 Available Tools from MCP Server:")
            print("=" * 50)
            for tool in tool_result.tools:
                print(f"🔧 {tool.name}: {tool.description}")
            
            # Test calculator tool
            print("\n🧪 Testing Calculator Tool:")
            print("=" * 50)
            result = await session.call_tool(name="calculate", arguments={"expression": "15 + 27"})
            print(f"calculate('15 + 27') = {result.content[0].text}")
            
            # Test current time tool
            print("\n🕐 Testing DateTime Tool:")
            print("=" * 50)
            result = await session.call_tool(name="get_current_time", arguments={})
            print(f"get_current_time() = {result.content[0].text}")

try:
    await test_mcp_server()
    print("\n✅ MCP Server is working correctly!")
except Exception as e:
    print(f"\n❌ Error testing MCP server: {e}")


📋 Available Tools from MCP Server:
🔧 calculate: 
    Evaluate a mathematical expression and return the result.

    Args:
        expression: A mathematical expression string (e.g., "2 + 2", "10 * 5 + 3")

    Returns:
        The result of the calculation as a string

    Examples:
        - calculate("5 + 3") -> "8"
        - calculate("10 * (5 + 2)") -> "70"
    
🔧 power: 
    Calculate base raised to the power of exponent.

    Args:
        base: The base number
        exponent: The exponent to raise the base to

    Returns:
        The result of base^exponent

    Examples:
        - power(2, 3) -> "8.0"
        - power(5, 2) -> "25.0"
    
🔧 write_file: 
    Write content to a file in the workspace.

    Args:
        filename: Name of the file to write
        content: Content to write to the file

    Returns:
        Success message or error message

    Examples:
        - write_file("notes.txt", "Hello World") -> "Successfully wrote to notes.txt"
    
🔧 read_file: 
    R

In [30]:
# Stop MCP server
try:
    mcp_proc.terminate()
    mcp_proc.wait(timeout=5)
    print("MCP Server stopped")
except:
    mcp_proc.kill()
    print("MCP Server killed")

MCP Server stopped


**Expected Output:**

You should see:
- 8 tools listed from the MCP server
- Successful calculation result (42)
- Current timestamp

This confirms the MCP server is working correctly before deployment.

## Step 5: Package the MCP Server

Now let's package the MCP server using AgentRun CLI.

In [11]:
# Package MCP server
!agentrun pack \
    -f mcp_server \
    --agent-name "mcp-utility-server" \
    --description "MCP server with calculator, file ops, and datetime tools" \
    --language "python" \
    --entrypoint "python main.py" \
    --port 8000 \
    --build-mode "local" \
    --verbose

INFO:agentrun.runtime.pack_runtime:Starting pack process for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/mcp_
server
DEBUG:agentrun.runtime.pack_runtime:Workspace structure validation passed: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/mcp_
server
DEBUG:agentrun.runtime.pack_runtime:Creating new metadata
DEBUG:agentrun.services.metadata_service:Saving metadata to: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/mcp_
server/agent_metadata.yaml
DEBUG:agentrun.runtime.pack_runtime:Applied option overrides: {'agent_name': 
'mcp-utility-server', 'language': 'python', 'entrypoint': 'python main.py', 
'port': 8000, 'build_mode': 'local'}
DEBUG:agentrun.runtime.pack_runtime:Found requirements.txt, processing 
dependencies
DEBUG:agentrun.runtime.pack_runtime:Generated Dockerfile: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/mcp_
server/Dockerfile
DEBUG:agentr

**Expected Output:**
```
✅ Successfully packaged agent: mcp-utility-server
📁 Workspace: /path/to/mcp_server
📄 Metadata: /path/to/mcp_server/agent_metadata.yaml
```

In [12]:
# View generated metadata
!cat mcp_server/agent_metadata.yaml

agent_name: mcp-utility-server
build_mode: local
description: MCP server with calculator, file ops, and datetime tools
entrypoint: python main.py
language: python
port: 8000
requirements_file: requirements.txt


## Step 6: Build the MCP Server Image

Build a Docker image for the MCP server.

In [28]:
# Build MCP server image
!agentrun build -f mcp_server --verbose

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.runtime.build_runtime:Starting build process for workspace: 
/Users/ym/Desktop/a

**Expected Output:**
```
✅ Successfully built agent image: mcp-utility-server:latest
🏷️  Tag: mcp-utility-server:latest
📏 Size: ~XXX MB
```

In [15]:
# Verify Docker image
!docker images | grep mcp-utility-server

mcp-utility-server:latest                                                                               e2998766c843        424MB             0B        


## Step 7: Publish MCP Server to Kubernetes

Deploy the MCP server to local Kubernetes cluster.

In [29]:
# Publish MCP server to Kubernetes
!agentrun publish \
    -f mcp_server \
    --version "v1.0.0" \
    --image-url "taoruiw/mcp-utility-server:latest" \
    --description "MCP server with utility tools" \
    --verbose \
    --node-port 30000 \
    --use-k8s

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_

**Expected Output:**
```
✅ Successfully published agent: mcp-utility-server
🌐 Service URL: http://localhost:XXXXX
📊 Kubernetes Deployment: mcp-utility-server in namespace agentrun
```

In [30]:
# Check MCP server status
!agentrun status -f mcp_server --use-k8s --verbose

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for namespace: agentrun
DEBUG:kubernetes.client.rest:response body: {"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8bdbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"managedFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1","time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.status_runtime:Checking agent status for workspace: /Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/mcp_server
DEBUG:agentrun.services.metadata_serv

In [31]:
# Get the MCP server service URL from metadata
import yaml

with open('mcp_server/agent_metadata.yaml', 'r') as f:
    mcp_metadata = yaml.safe_load(f)

mcp_service_url = mcp_metadata.get('k8s_deployment', {}).get('service_url', 'http://localhost:30000')
mcp_server_endpoint = f"{mcp_service_url}/mcp"

print(f"MCP Server Service URL: {mcp_service_url}")
print(f"MCP Server Endpoint: {mcp_server_endpoint}")

# Save for agent configuration
with open('.mcp_server_url.txt', 'w') as f:
    f.write(mcp_server_endpoint)

MCP Server Service URL: http://localhost:30000
MCP Server Endpoint: http://localhost:30000/mcp


## Step 8: Test Deployed MCP Server

Verify the MCP server is working in Kubernetes.

In [38]:
# Test deployed MCP server
import asyncio
from datetime import timedelta
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def test_deployed_mcp():
    # Read MCP server URL
    with open('.mcp_server_url.txt', 'r') as f:
        mcp_url = f.read().strip()
    
    print(f"Testing MCP server at: {mcp_url}")
    headers = {}
    
    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=30), terminate_on_close=False) as (
            read_stream, write_stream, _
        ):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                
                # List tools
                tool_result = await session.list_tools()
                print(f"\n✅ MCP Server is running with {len(tool_result.tools)} tools")
                print("\nAvailable tools:")
                for tool in tool_result.tools:
                    print(f"  - {tool.name}")
    except Exception as e:
        print(f"\n❌ Error connecting to MCP server: {e}")
        print("Make sure the MCP server is deployed and accessible")

await test_deployed_mcp()

Testing MCP server at: http://localhost:30000/mcp

✅ MCP Server is running with 8 tools

Available tools:
  - calculate
  - power
  - write_file
  - read_file
  - list_files
  - get_current_time
  - format_timestamp
  - server_info


## Step 9: Configure Agent with MCP Server URL

Before packaging the agent, we need to configure it with the MCP server URL.

In [ ]:
# Create agent environment configuration
with open('.mcp_server_url.txt', 'r') as f:
    mcp_url = f.read().strip()

# For Kubernetes deployment, we'll use the service name
# The service name will be mcp-utility-server in the agentrun namespace
k8s_mcp_url = "http://mcp-utility-server.agentrun.svc.cluster.local:8000/mcp"

print(f"Local MCP URL: {mcp_url}")
print(f"Kubernetes MCP URL: {k8s_mcp_url}")
print("\nThe agent will use the Kubernetes service URL when deployed to k8s")

Local MCP URL: http://localhost:30000/mcp
Kubernetes MCP URL: http://mcp-utility-server.agentrun.svc.cluster.local:30000/mcp

The agent will use the Kubernetes service URL when deployed to k8s


## Step 10: Package the LangChain Agent

Now let's package the LangChain agent.

In [7]:
# Package the agent
!agentrun pack \
    -f agent \
    --agent-name "langchain-mcp-agent" \
    --description "LangChain agent that uses MCP server tools" \
    --language "python" \
    --entrypoint "python main.py" \
    --port 8080 \
    --build-mode "local" \
    --verbose

INFO:agentrun.runtime.pack_runtime:Starting pack process for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.runtime.pack_runtime:Workspace structure validation passed: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.services.metadata_service:Loading metadata from: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t/agent_metadata.yaml
DEBUG:agentrun.runtime.pack_runtime:Loaded existing metadata
DEBUG:agentrun.runtime.pack_runtime:Applied option overrides: {'agent_name': 
'langchain-mcp-agent', 'language': 'python', 'entrypoint': 'python main.py', 
'port': 8080, 'build_mode': 'local'}
DEBUG:agentrun.runtime.pack_runtime:Found requirements.txt, processing 
dependencies
DEBUG:agentrun.runtime.pack_runtime:Dockerfile already exists, skipping 
generation
DEBUG:agentrun.services.metadata_service:Loading metadata from: 
/Users/ym/Desktop/agentcube/c

## Step 11: Build the Agent Image

Build a Docker image for the LangChain agent.

In [1]:
# Build agent image
!agentrun build -f agent --verbose

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.runtime.build_runtime:Starting build process for workspace: 
/Users/ym/Desktop/a

In [34]:
# Verify Docker image
!docker images | grep langchain-mcp-agent

langchain-mcp-agent:latest                                                                              e8cc26c18fc7        528MB             0B        


## Step 12: Publish Agent to Kubernetes

Deploy the LangChain agent to Kubernetes with MCP server URL configured.

In [2]:
# Publish agent to Kubernetes
# Note: The agent will automatically connect to the MCP server via Kubernetes service discovery
!agentrun publish \
    -f agent \
    --version "v1.0.0" \
    --image-url "taoruiw/langchain-mcp-agent:latest" \
    --description "LangChain agent with MCP server integration" \
    --verbose \
    --use-k8s

DEBUG:docker.utils.config:Trying paths: ['/Users/ym/.docker/config.json', 
'/Users/ym/.dockercfg']
DEBUG:docker.utils.config:Found file at path: /Users/ym/.docker/config.json
DEBUG:docker.auth:Found 'auths' section
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/ is absent. Client 
might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/access-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Auth data for https://index.docker.io/v1/refresh-token is 
absent. Client might be using a credentials store instead.
DEBUG:docker.auth:Found 'credsStore' section
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/1.1" 200 
None
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.52/_ping HTTP/1.1" 
200 None
INFO:agentrun.services.docker_service:Successfully connected to Docker daemon
INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_

**Note**: In a production environment, you would set environment variables in the Kubernetes deployment to configure the MCP_SERVER_URL. For this tutorial, the agent defaults to connecting to the MCP server service within the cluster.

In [3]:
# Check agent status
!agentrun status -f agent --use-k8s --verbose

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for namespace: agentrun
DEBUG:kubernetes.client.rest:response body: {"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8bdbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"managedFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1","time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.status_runtime:Checking agent status for workspace: /Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agent
DEBUG:agentrun.services.metadata_service:L

## Step 13: Test the Integrated System

Now let's test the complete system: agent communicating with MCP server.

In [4]:
# Test agent health
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "health check"}' \
    --verbose \
    --use-k8s

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for 
namespace: agentrun
DEBUG:kubernetes.client.rest:response body: 
{"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8b
dbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"
2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"manag
edFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1"
,"time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":
{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers
":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.services.meta

In [5]:
# Test calculator tool through agent
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "Calculate 25 multiplied by 4"}' \
    --verbose \
    --use-k8s

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for 
namespace: agentrun
DEBUG:kubernetes.client.rest:response body: 
{"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8b
dbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"
2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"manag
edFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1"
,"time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":
{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers
":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.services.meta

**Expected Output (Mock Mode):**
```
✅ Successfully invoked agent
📤 Response: [Mock Mode] Received prompt: 'Calculate 25 multiplied by 4'.
Available tools from MCP server: calculate, power, write_file, read_file, 
list_files, get_current_time, format_timestamp, server_info.
Set OPENAI_API_KEY environment variable for real agent responses.
```

**Expected Output (Production Mode with OpenAI API key):**
```
✅ Successfully invoked agent
📤 Response: The result of 25 multiplied by 4 is 100.
```

In [7]:
# Test datetime tool
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "What is the current time?"}' \
    --verbose \
    --use-k8s

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for 
namespace: agentrun
DEBUG:kubernetes.client.rest:response body: 
{"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8b
dbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"
2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"manag
edFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1"
,"time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":
{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers
":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.services.meta

In [6]:
# Test file operations
!agentrun invoke \
    -f agent \
    --payload '{"prompt": "Write Hello from LangChain agent to a file called greeting.txt"}' \
    --verbose \
    --use-k8s

INFO:agentrun.services.k8s_provider:Loaded local Kubernetes config
INFO:agentrun.services.k8s_provider:Kubernetes provider initialized for 
namespace: agentrun
DEBUG:kubernetes.client.rest:response body: 
{"kind":"Namespace","apiVersion":"v1","metadata":{"name":"agentrun","uid":"40e8b
dbd-a04c-4884-b7ea-7883ef65df9b","resourceVersion":"16799","creationTimestamp":"
2025-11-18T12:02:51Z","labels":{"kubernetes.io/metadata.name":"agentrun"},"manag
edFields":[{"manager":"OpenAPI-Generator","operation":"Update","apiVersion":"v1"
,"time":"2025-11-18T12:02:51Z","fieldsType":"FieldsV1","fieldsV1":{"f:metadata":
{"f:labels":{".":{},"f:kubernetes.io/metadata.name":{}}}}}]},"spec":{"finalizers
":["kubernetes"]},"status":{"phase":"Active"}}

DEBUG:agentrun.services.k8s_provider:Namespace agentrun already exists
INFO:agentrun.runtime.invoke_runtime:Starting agent invocation for workspace: 
/Users/ym/Desktop/agentcube/cmd/agentrun/03-langchain-agent-with-mcp-server/agen
t
DEBUG:agentrun.services.meta

## Step 14: Verify Kubernetes Deployments

Let's verify both services are running properly in Kubernetes.

In [8]:
# Check Kubernetes pods
!kubectl get pods -n agentrun

NAME                                   READY   STATUS    RESTARTS   AGE
langchain-mcp-agent-775c8657bc-lmwnx   1/1     Running   0          68m
mcp-utility-server-8c6578fb6-n4r6m     1/1     Running   0          105m


In [9]:
# Check Kubernetes services
!kubectl get services -n agentrun

NAME                  TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
langchain-mcp-agent   NodePort   10.96.114.200   <none>        8080:32754/TCP   68m
mcp-utility-server    NodePort   10.111.85.177   <none>        8000:30000/TCP   105m


In [10]:
# Check deployments
!kubectl get deployments -n agentrun

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
langchain-mcp-agent   1/1     1            1           68m
mcp-utility-server    1/1     1            1           105m


**Expected Output:**

You should see:
- 2 pods running (mcp-utility-server and langchain-mcp-agent)
- 2 services exposed
- 2 deployments with 1/1 replicas ready

This confirms both components are deployed and running in Kubernetes.

## Step 15: Understanding the Architecture

### Microservices Architecture Benefits

This distributed architecture provides several advantages:

**1. Separation of Concerns**
- MCP Server: Focuses solely on providing tools
- LangChain Agent: Focuses on orchestration and reasoning

**2. Independent Scaling**
- Scale the MCP server if tools are heavily used
- Scale the agent if there are many user requests
- Each component scales based on its own load

**3. Reusability**
- Multiple agents can connect to the same MCP server
- One agent can connect to multiple MCP servers
- Tools are centralized and reusable

**4. Technology Flexibility**
- MCP server could be written in any language
- Agent framework can be changed without affecting tools
- Each service can use different technology stacks

**5. Fault Isolation**
- If MCP server fails, agent can handle it gracefully
- If agent fails, MCP server continues running
- Independent health checks and monitoring

### MCP Protocol Benefits

**Standardized Communication:**
- Well-defined protocol for tool discovery
- Consistent tool invocation format
- Support for streaming and stateless operation

**LangChain Integration:**
- Seamless conversion to LangChain tools
- Compatible with existing LangChain ecosystem
- Works with any LangChain-compatible LLM

## Step 16: Advanced Usage (Optional)

### Adding OpenAI API Key for Production Mode

To use the agent in production mode with real reasoning:

1. Get an OpenAI API key from https://platform.openai.com/api-keys
2. Update the Kubernetes deployment with the API key:

```bash
kubectl set env deployment/langchain-mcp-agent \
    OPENAI_API_KEY=your-api-key-here \
    -n agentrun
```

3. The agent will automatically restart and use OpenAI for reasoning

### Scaling the Services

Scale the MCP server:
```bash
kubectl scale deployment mcp-utility-server --replicas=3 -n agentrun
```

Scale the agent:
```bash
kubectl scale deployment langchain-mcp-agent --replicas=2 -n agentrun
```

### Adding More Tools to MCP Server

To add new tools:
1. Add tool functions to `mcp_server/main.py`
2. Decorate with `@mcp.tool()`
3. Rebuild and redeploy: `agentrun build` and `agentrun publish`
4. Restart the agent to discover new tools

## Summary and Best Practices

### What We Learned

In this tutorial, we covered:

1. ✅ **Created an MCP server** with 8 utility tools
2. ✅ **Built a LangChain agent** that integrates with MCP server
3. ✅ **Deployed both services** separately to Kubernetes
4. ✅ **Tested the integrated system** with various tool invocations
5. ✅ **Understood microservices architecture** for AI agents

### Best Practices

#### 1. MCP Server Design
- **Group related tools**: Organize tools by category (calculator, file ops, etc.)
- **Use stateless HTTP**: Required for AgentRun deployment
- **Provide clear descriptions**: Helps agents understand tool purpose
- **Handle errors gracefully**: Return informative error messages
- **Keep tools focused**: Each tool should do one thing well

#### 2. LangChain Agent Integration
- **Implement mock mode**: Allows testing without API keys
- **Cache tool discovery**: Don't rediscover tools on every request
- **Handle connection failures**: Gracefully handle MCP server unavailability
- **Use environment variables**: For configuration (MCP URL, API keys)
- **Add health checks**: Monitor agent and MCP server health

#### 3. Deployment
- **Deploy services separately**: Allows independent scaling
- **Use Kubernetes service discovery**: For service-to-service communication
- **Set resource limits**: Prevent resource exhaustion
- **Implement monitoring**: Track both services independently
- **Use secrets for API keys**: Don't hardcode sensitive data

#### 4. Testing
- **Test locally first**: Before deploying to Kubernetes
- **Test tools individually**: Verify each tool works correctly
- **Test agent integration**: Ensure agent can call all tools
- **Test failure scenarios**: What happens if MCP server is down?

### Common Patterns

**1. Multi-Agent Systems**
- Multiple agents sharing one MCP server
- Reduces duplication and centralizes tools

**2. Agent Chains**
- One agent calls another agent
- Each agent has different capabilities via different MCP servers

**3. Tool Composition**
- Multiple MCP servers providing different tool categories
- Agent aggregates tools from multiple sources

### Next Steps

Now that you understand MCP server integration, you can:

1. **Add custom tools** to the MCP server for your use case
2. **Create specialized agents** for different domains
3. **Build multi-agent systems** with shared tool infrastructure
4. **Implement authentication** for secure tool access
5. **Add observability** with logging and metrics
6. **Deploy to production** cloud Kubernetes clusters

### Additional Resources

- [MCP Protocol Specification](https://modelcontextprotocol.io/)
- [LangChain Documentation](https://python.langchain.com/)
- [AgentRun CLI Documentation](../../QUICKSTART.md)
- [FastMCP Documentation](https://github.com/modelcontextprotocol/fastmcp)
- [AgentCube Project](https://github.com/volcano-sh/agentcube)

## Cleanup (Optional)

To clean up the resources created in this tutorial:

In [ ]:
# Delete Kubernetes deployments
!kubectl delete deployment langchain-mcp-agent -n agentrun
!kubectl delete deployment mcp-utility-server -n agentrun

# Delete services
!kubectl delete service langchain-mcp-agent -n agentrun
!kubectl delete service mcp-utility-server -n agentrun

In [ ]:
# Remove Docker images (optional)
!docker rmi langchain-mcp-agent:latest
!docker rmi mcp-utility-server:latest

In [ ]:
# Remove generated files (optional)
!rm -f agent/agent_metadata.yaml agent/Dockerfile
!rm -f mcp_server/agent_metadata.yaml mcp_server/Dockerfile
!rm -f .mcp_server_url.txt

# Congratulations! 🎉

You've successfully completed the LangChain Agent with MCP Server tutorial!

You now know how to:
- ✅ Create MCP servers with custom tools
- ✅ Integrate MCP servers with LangChain agents
- ✅ Deploy distributed agent systems using AgentRun CLI
- ✅ Build microservices architectures for AI agents

Happy building! 🚀